# Nonlinear Least Squares for 3D Point Intersection
# ===============================================
# This script computes the 3D coordinates of a point P using nonlinear least squares,
# given distance measurements from five known stations (A, B, C, D, E).

In [1]:
import numpy as np

# Step 1: Define Input Data
# -------------------------

In [2]:
# Station coordinates (X, Y, Z) in meters
X = np.array([2742.87, 2751.59, 2738.08, 2715.23, 2724.96])
Y = np.array([91874.15, 91872.49, 91886.95, 91852.36, 91882.74])
Z = np.array([510.56, 513.76, 518.11, 515.39, 516.28])

In [3]:

# Observed distances from stations to point P (in meters)
D = np.array([3.71, 6.17, 15.90, 37.82, 22.90])


In [4]:
# Initial guess for P's coordinates
Xp, Yp, Zp = 2746.0, 91874.0, 512.0

# Step 2: Nonlinear Least Squares
# ------------------------------

In [5]:
# Initialize corrections and iteration counter
dxp, dyp, dzp = 1.0, 1.0, 1.0
it = 0
threshold = 1e-5  # Convergence threshold (meters)


In [6]:
while np.max(np.abs([dxp, dyp, dzp])) >= threshold:
    it += 1
    
    # Compute distances from current estimate of P to stations
    Dc = np.sqrt((X - Xp)**2 + (Y - Yp)**2 + (Z - Zp)**2)
    
    # Residuals: observed - computed distances
    l = D - Dc
    
    # Design matrix A (Jacobian of distances w.r.t. Xp, Yp, Zp)
    A = np.column_stack([
        (Xp - X) / Dc,  # dD/dXp
        (Yp - Y) / Dc,  # dD/dYp
        (Zp - Z) / Dc   # dD/dZp
    ])
    
    # Solve for corrections: (A^T A)^(-1) A^T l
    delta = np.linalg.solve(A.T @ A, A.T @ l)
    dxp, dyp, dzp = delta
    
    # Update coordinates
    Xp += dxp
    Yp += dyp
    Zp += dzp


# Step 3: Compute Covariance Matrix
# --------------------------------

In [7]:
# Recompute Dc and A at final estimate
Dc = np.sqrt((X - Xp)**2 + (Y - Yp)**2 + (Z - Zp)**2)
A = np.column_stack([
    (Xp - X) / Dc,
    (Yp - Y) / Dc,
    (Zp - Z) / Dc
])

# Residuals
v = D - Dc

# Variance of unit weight (sigma_0^2)
n = len(D)  # Number of observations
u = 3       # Number of unknowns (Xp, Yp, Zp)
sigma_0_squared = (v.T @ v) / (n - u)

# Covariance matrix: sigma_0^2 * (A^T A)^(-1)
cov_matrix = sigma_0_squared * np.linalg.inv(A.T @ A)
sigma_p = np.sqrt(np.diag(cov_matrix))

# Step 4: Display Results
# -----------------------

In [8]:
print(f"Number of iterations: {it}")
print(f"\nEstimated coordinates of P:")
print(f"X_P = {Xp:.4f} m")
print(f"Y_P = {Yp:.4f} m")
print(f"Z_P = {Zp:.4f} m")

print(f"\nStandard deviations:")
print(f"σ_X = {sigma_p[0]:.4f} m")
print(f"σ_Y = {sigma_p[1]:.4f} m")
print(f"σ_Z = {sigma_p[2]:.4f} m")

print(f"\nFinal residuals (observed - computed distances):")
for i, res in enumerate(v):
    print(f"Station {chr(65+i)}: {res:.4f} m")

print(f"\nVariance of unit weight (σ_0²): {sigma_0_squared:.4f}")

Number of iterations: 6

Estimated coordinates of P:
X_P = 2745.9040 m
Y_P = 91874.2967 m
Z_P = 512.5890 m

Standard deviations:
σ_X = 0.0479 m
σ_Y = 0.0826 m
σ_Z = 0.1266 m

Final residuals (observed - computed distances):
Station A: 0.0571 m
Station B: 0.0900 m
Station C: 0.0317 m
Station D: 0.0052 m
Station E: 0.0185 m

Variance of unit weight (σ_0²): 0.0064
